# Trajectory Optimization Example

Construct a trajectory with two states (pose + velocity). Lock the first state (pose + velocity) and only the velocity of the second state, and optimize the pose of the second state.


In [1]:
import numpy as np

np.set_printoptions(6, suppress=True)

from pylgmath import Transformation
from pysteam.trajectory import Time
from pysteam.trajectory.const_vel import Interface as TrajectoryInterface
from pysteam.problem import OptimizationProblem
from pysteam.solver import GaussNewtonSolver, DoglegGaussNewtonSolver
from pysteam.evaluable.se3 import SE3StateVar
from pysteam.evaluable.vspace import VSpaceStateVar

Set up the trajectory that uses a constant velocity motion prior:


In [2]:
# states with initial conditions and associated timestamps
NUM_STATES = 2
DT = 1
states = [(DT * i, Transformation(), i * np.ones((6, 1))) for i in range(NUM_STATES)]

# wrap states with corresponding steam state variables (no copy!)
state_vars = [
    (t, SE3StateVar(T_vi), VSpaceStateVar(w_iv_inv)) for t, T_vi, w_iv_inv in states
]
state_vars[0][1].locked = True  # lock first pose
state_vars[0][2].locked = True  # lock first velocity
state_vars[1][2].locked = True  # lock second velocity

# construct a trajectory
qcd = np.ones(6)
traj = TrajectoryInterface(qcd=qcd)
for t, T_vi, w_iv_inv in state_vars:
    traj.add_knot(time=Time(t), T_k0=T_vi, w_0k_ink=w_iv_inv)

Construct the optimization problem and solve:


In [3]:
opt_prob = OptimizationProblem()
opt_prob.add_state_var(*[v for state_var in state_vars for v in state_var[1:]])
opt_prob.add_cost_term(*traj.get_prior_cost_terms())

solver = DoglegGaussNewtonSolver(opt_prob, verbose=True)
solver.optimize()

Begin Optimization
------------------
Number of States:  1
Number of Cost Terms:  1
Initial Cost:  12.000000000000002
Iteration:    1  -  Cost:     3.0000  -  TR Shrink:  0.000  -  AvP Ratio:  1.000  -  Dogleg Segment: Gauss Newton   
Iteration:    2  -  Cost:     3.0000  -  TR Shrink: 50.000  -  AvP Ratio:  0.000  -  Dogleg Segment: Gauss Newton   
Termination Cause:  CONVERGED ZERO GRADIENT


Print out results:


In [4]:
print("First Pose:                 \n", states[0][1])
print("Second Pose:                \n", states[1][1])
print("First Vel:                  \n", states[0][2].squeeze())
print("Second Vel:                 \n", states[1][2].squeeze())
print(
    "Interp. Vel (t=t0+0.5*DT):  \n",
    traj.get_velocity_interpolator(Time(states[0][0]) + Time(secs=0.5 * DT))
    .evaluate()
    .squeeze(),
)

First Pose:                 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Second Pose:                
 [[ 0.76524  -0.322422  0.557183  0.5     ]
 [ 0.557183  0.76524  -0.322422  0.5     ]
 [-0.322422  0.557183  0.76524   0.5     ]
 [ 0.        0.        0.        1.      ]]
First Vel:                  
 [0. 0. 0. 0. 0. 0.]
Second Vel:                 
 [1. 1. 1. 1. 1. 1.]
Interp. Vel (t=t0+0.5*DT):  
 [0.5 0.5 0.5 0.5 0.5 0.5]
